In [62]:
from bs4 import BeautifulSoup
import re
import json
from datasets import Dataset
import torch
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import pandas as pd 
import yfinance as yf 
from yahoofinancials import YahooFinancials
from datetime import datetime

In [63]:
gps_soup = BeautifulSoup(open(r"C:\Users\User\Desktop\thesis_final\stock_price_and_news\stock_news\dataset\gps_final\gps_jan_2007.html", encoding="utf8"), 
                     "html.parser")
tsla_soup = BeautifulSoup(open(r"C:\Users\User\Desktop\thesis_final\stock_price_and_news\stock_news\dataset\tsla_final\tsla_2017_jan.html", encoding="utf8"), 
                     "html.parser")
jpm_soup = BeautifulSoup(open(r"C:\Users\User\Desktop\thesis_final\stock_price_and_news\stock_news\dataset\jpm_final\jpm_2012_jan.html", encoding="utf8"), 
                     "html.parser")



# getting timestamps

In [64]:
gps_timestamp = []
for e in gps_soup.find_all("span", {"class": "article__timestamp"}):
    gps_timestamp.append(e.get_text())
tsla_timestamp = []
for e in tsla_soup.find_all("span", {"class": "article__timestamp"}):
    tsla_timestamp.append(e.get_text())
jpm_timestamp = []
for e in jpm_soup.find_all("span", {"class": "article__timestamp"}):
    jpm_timestamp.append(e.get_text())

# getting text

In [65]:
tsla_file = []
for tsla_text in tsla_soup.find_all("h3"):
    tsla_file.append(tsla_text.get_text())
gps_file = []
for gps_text in gps_soup.find_all("h3"):
    gps_file.append(gps_text.get_text())
jpm_file = []
for jpm_text in jpm_soup.find_all("h3"):
    jpm_file.append(jpm_text.get_text())

In [66]:
jpm_file[-10:]

['\n\n                            \n                            DCM Advisors, LLC Buys Apple Inc, Microsoft Corp, Alphabet Inc, Sells MetLife Inc, Qorvo Inc, ...\n                        \n',
 '\n\n                            \n                            Down 38% From Its High, Is This Fintech a Buy?\n                        \n',
 '\n\n                            \n                            1 Key Metric Shows How Wells Fargo Can Recover\n                        \n',
 "\n\n                            \n                            Here's Another Reason Why Shopify Is an Unstoppable E-Commerce Stock to Buy in 2022\n                        \n",
 '\n\n                            \n                            ING Groep NV Buys Amazon.com Inc, Micron Technology Inc, JPMorgan Chase, Sells Enterprise ...\n                        \n',
 '\n\n                            \n                            Goodbye Accommodating Fed, JPMorgan Could Rise Sharply\n                        \n',
 '\n\n     

In [67]:
gps_clean1 = []
for gps_text1 in gps_file:
    gps_clean1.append(gps_text1.replace("\n", ""))
tsla_clean1 = []
for tsla_text1 in tsla_file:
    tsla_clean1.append(tsla_text1.replace("\n", ""))
jpm_clean1 = []
for jpm_text1 in jpm_file:
    jpm_clean1.append(jpm_text1.replace("\n", ""))

In [68]:
gps_remove = []
for e in gps_clean1:
    if e not in ('Intel announces $5.4 billion deal to buy Tower Semiconductor',
 'Intel announces $5.4 billion deal to buy Tower Semiconductor',
 'The most common excuses for not buying an electric vehicle are mostly unfounded',
 'Ray Dalio’s Bridgewater boosts its bets on China',
 'What is the Alfa Romeo Tonale, and why does it come with a non-fungible token?',
 'What is a multicurrency account, and should you get one?',
 '‘Those years passed, and here I am.’ What it’s like to live to 100.',
 "Barron'sInvestors, Beware: These Funds Could Pose a 'Flight Risk' for Junk Bond Markets",
 'No Recent Tickers',
 'Overview'):
        gps_remove.append(e)
tsla_remove = []
for e in tsla_clean1:
    if e not in ('Virgin Galactic stock jumps 30% as space-tourism tickets to open soon',
 "Barron'sGap Stock Gets a Downgrade. 2 Big Issues Worry the Analyst.",
 "Meta changes News Feed to 'Feed'",
 'Avis stock sinks as bears focus on revenue per day disappointment over earnings beat',
 "BreakingBiden to give 'update' on Russia and Ukraine, White House says",
 'OpinionDon’t be fearful: Stock market investors could ultimately benefit from an economic recession',
 'This Wall Street pro foresaw high inflation. Here’s what he thinks the Fed has to do',
 'Senate Republicans plan to boycott committee vote on Powell and four other Biden Fed picks',
 "Biden to say U.S. 'remains open to high-level diplomacy' with Russia ",
 "Barron'sSenate Republicans to Delay Panel Vote on Raskin Fed Nomination",
 'Biden to speak on Russia and Ukraine situation at 3:30 p.m. Eastern, White House says',
 "Barron'sSenate Republicans to Delay Panel Vote on Raskin Fed Nomination",
 'Why the Russia-Ukraine crisis may make food-price inflation even worse',
 'PSEG boosts dividends, as implied yield remains well more than double that of the S&P 500',
 "Barron'sVirgin Galactic Opens Reservations for Space. How to Book a Flight.",
 'PSEG raises quarterly dividend by nearly 6% to 54 cents a share',
 'Lower house of parliament calls on Putin to recognize pro-Russia eastern regions of Ukraine as legitimate republics',
 "Barron'sGE Stock Is Soaring. The Annual Report Has Provided a Boost.",
 'Cannabis banking firm Safe Harbor going public via SPAC deal',
 'Over 112 million people watched the Super Bowl — up 14% from last year',
 "Barron'sStanley Druckenmiller Got Bullish on Chevron. Should You Be?",
 "Barron'sFisker Is Taking Orders for Its New EV. Starting Price Is Below $30,000. ",
 'More states are preventing local governments from requiring paid sick leave —  but low-wage workers end up paying the price',
 '‘It’s really hard for locals to compete’: Out-of-towners have one major advantage over locals in America’s hottest housing markets',
 'Cannabis beverage company Cann gets backing from Green Thumb Industries and celebrity investors',
 'Over 112 million people watched the Super Bowl — up 14% from last year',
 'Prince Andrew reaches settlement with Virginia Giuffre in sexual abuse lawsuit',
 'Over 112 million people watched the Super Bowl — up 14% from last year',
 'CorrectionCanada announces sweeping measures to end trucker protests against virus restrictions, as U.S. COVID death toll tops 922,000',
 "Barron'sWhy Now Is the Time to Dive Into Tech Stocks",
 "Barron'sAir-Taxi Stock Joby Is Soaring. An Airline Deal Shows the Sector's Potential.",
 "Barron'sStocks Jump After Russia Says Troops Are Pulling Back From Ukraine ",
 "Amazon's Just Walk Out technology lands at NYC's LaGuardia Airport",
 "Barron'sWhy Apple Is Outpacing Netflix, Microsoft, and the Rest of Big Tech",
 'Sandy Hook families settle with gun maker Remington for $73 million',
 "BreakingBoeing, Nike share gains lead Dow's 382-point rally",
 "Barron'sTesla Stock Needs to Break Free From Putin and Powell",
 'Fisker to start reservations for second EV, with production slated for November',
 "Barron'sMastercard Is Hiring to Expand Crypto and ESG Consulting.",
 'Dow jumps over 400 points as Russia-Ukraine tensions ease',
 "Barron'sTesla Stock Has Catalysts. It Needs to Break Free From Putin and Powell.",
 'OpinionThis fail-safe protection against inflation is back',
 "Barron'sNvidia's Earnings Could Be a Big Win. What to Expect.",
 "Barron'sCoinbase Plans to Hire 2,000 Employees Because of  'Enormous' Web3 Opportunities",
 'Amazon lifts mask requirement for fully vaccinated workers',
 "Barron'sWhat a Monster, Constellation Merger Might Mean and Why Coca-Cola Could Be Key",
 "Barron'sBitcoin Is Bouncing. Russia Is Helping. ",
 'Pentagon report shows contractor mergers hurt U.S. economy and national security',
 'I have $1 million and a $100,000-a-year pension. My girlfriend and I are getting married, but she’s against a prenup. What should I do?',
 'Iamgold upgraded to outperform from market perform at BMO Capital',
 'Petroshale started at outperform at BMO Capital',
 'This is what you’re getting wrong about ESG ratings',
 'Gap stock price target cut to $14 from $26 at BofA Securities',
 'Gap downgraded to underperform from neutral at BofA Securities',
 'Oil drops more than 4% from a 7-year high as Russia says some troops returning to base',
 'DTE Energy double upgraded to buy from underperform at BofA Securities',
 'Canada announces sweeping measures to end trucker protests against virus restrictions, as U.S. COVID death toll tops 922,000',
 "Barron'sFamed Spelling Manor in Los Angeles Lists for $165 Million",
 '‘I offered a $10,000 reward for the person who introduces me to my future partner’: I’m 34, single and disabled. How do I meet the woman of my dreams?',
 'OpinionInflation is making some seniors choose between food and medications — what you can do',
 'No Recent Tickers',
 'Overview'):
        tsla_remove.append(e)
jpm_remove = []
for e in jpm_clean1:
    if e not in ("Barron'sStocks Wobble With Fed Minutes Ahead",
 '',
 "Barron'sUpstart Stock Surges on Strong Revenue Forecast and Buyback Plans",
 'Personal loans and taxes: Don’t let this ‘unpleasant surprise’ happen to you if you take out a personal loan',
 'Midcareer millennials now have negotiating power—how to get the job you want, on the terms you want',
 'The 2022 Ford Mustang Mach-E—practical and high tech, with performance that lives up to its name',
 'Trying to build a business in midlife? Here are 7 tips from 2 marketing masters',
 "Barron'sNvidia's Earnings Are Today. Gaming and Data Center Are the Money Makers.",
 "Barron'sThe largest Fancy Vivid Blue Diamond to Hit Auction for $48 Million",
 'UpdatedEricsson shares slump as telecommunications firm says it may have made payments to Islamic State',
 'Ericsson shares slump as telecommunications firm says it may have made payments to Islamic State',
 'With a tiger and Jamie Dimon, JPMorgan enters the metaverse',
 'Ericsson discloses ‘serious breach of compliance rules’ in Iraq dealings',
 'The end of the pandemic may be in sight, says Moderna CEO',
 "Barron'sPublicly Traded Companies Are Vanishing. Does Private Equity Have an Edge?",
 "Barron'sWynn Stock Falls After Quarterly Loss Wider Than Estimates",
 "Barron'sThe ARK Innovation Selloff Looks Like the Dot-Com Bust. History Says It Gets Worse.",
 "Barron'sWynn Stock Falls After Quarterly Loss Wider Than Estimates",
 "Barron'sCisco Earnings Are Coming. Watch Hardware Production. ",
 'No Recent Tickers',
 'Overview'):
        jpm_remove.append(e)


In [69]:
len(tsla_remove)

7603

In [70]:
gps_space = []
for e in gps_remove:
    gps_space.append(e.strip())
jpm_space = []
for e in jpm_remove:
    jpm_space.append(e.strip())
tsla_space = []
for e in tsla_remove:
    tsla_space.append(e.strip())

In [71]:
gps_fullstop = []
for e in gps_space:
    gps_fullstop.append(e.replace(".",""))
jpm_fullstop = []
for e in jpm_space:
    jpm_fullstop.append(e.replace(".",""))
tsla_fullstop = []
for e in tsla_space:
    tsla_fullstop.append(e.replace(".",""))

In [72]:
gps_q = []
for e in gps_fullstop:
    gps_q.append(e.replace("?",""))
jpm_q = []
for e in jpm_fullstop:
    jpm_q.append(e.replace("?",""))
tsla_q = []
for e in tsla_fullstop:
    tsla_q.append(e.replace("?",""))


In [73]:
gps_colon = []
for e in gps_q:
    gps_colon.append(e.replace(":",""))
jpm_colon = []
for e in jpm_q:
    jpm_colon.append(e.replace(":",""))
tsla_colon = []
for e in tsla_q:
    tsla_colon.append(e.replace(":",""))

In [74]:
gps_comma = []
for e in gps_colon:
    gps_comma.append(e.replace(",",""))
jpm_comma = []
for e in jpm_colon:
    jpm_comma.append(e.replace(",",""))
tsla_comma = []
for e in tsla_colon:
    tsla_comma.append(e.replace(",",""))


In [75]:
gps_percent = []
for e in gps_comma:
    gps_percent.append(e.replace("%","percent"))
jpm_percent = []
for e in jpm_comma:
    jpm_percent.append(e.replace("%","percent"))
tsla_percent = []
for e in tsla_comma:
    tsla_percent.append(e.replace("%","percent"))

In [76]:
tsla_lower = []
for i in range(len(tsla_percent)):
    tsla_lower.append(tsla_percent[i].lower())
jpm_lower = []
for i in range(len(jpm_percent)):
    jpm_lower.append(jpm_percent[i].lower())
gps_lower = []
for i in range(len(gps_percent)):
    gps_lower.append(gps_percent[i].lower())

In [77]:
len(tsla_lower)

7603

In [78]:
jpm_pain = []
for e in jpm_lower:
    if e not in ( 'marketwatch',
 'company',
 'dow jones network'):
        jpm_pain.append(e)
gps_pain = []
for e in gps_lower:
    if e not in ( 'marketwatch',
 'company',
 'dow jones network'):
        gps_pain.append(e)
tsla_pain = []
for e in tsla_lower:
    if e not in ( 'marketwatch',
 'company',
 'dow jones network'):
        tsla_pain.append(e)


In [79]:
len(jpm_lower)

7183

In [80]:
tsla_pain[-10:]

['eulav asset management buys uber technologies inc tesla inc roku inc sells ball corp ',
 "big tech's big returns and sports business in the spotlight",
 '7 stocks to love for a long long time',
 'coatue management llc buys rivian automotive inc pfizer inc visa inc sells doordash inc ',
 'myriad asset management ltd buys tesla inc mirion technologies inc enphase energy inc sells ',
 '7 top stocks for february ready for v-shaped bounce',
 'why tesla stock raced ahead on monday',
 'twilio stock has disaster stats but it also has great potential',
 'nio’s results should show the tesla competitor is real competition',
 '3 top ev stocks to buy during the market correction']

In [81]:
device = 'cpu'

## import pretrained finbert from huggingface

In [82]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

#model = model.to(device)

## Create softmax to rescale data

In [83]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x, axis=1)[:, None])
    return e_x / np.sum(e_x, axis=1)[:, None]

# classify and rescale sentiment output

## Gap

In [84]:
label_dict = {0: 'positive', 1: 'negative', 2: 'neutral'}

gps_out = []
gps_score = []
for i in range(0, len(gps_pain), 10):
    tokenized_gps = tokenizer(gps_pain[i:i+10], truncation = True, padding = True, return_tensors = "pt")
    gps_output = model(tokenized_gps['input_ids'])
    soft_logits_gps = softmax(np.array(gps_output.logits.detach().cpu()))
    sentiment_score_gps = soft_logits_gps[:,0]-soft_logits_gps[:,1]
    gps_pred = np.squeeze(np.argmax(soft_logits_gps, axis = 1))
    gps_out.extend(gps_pred)
    gps_score.extend(sentiment_score_gps)
#print(gps_out) 
print(sentiment_score_gps)

[ 0.08874428 -0.1351273   0.03656105  0.05744809  0.06736011 -0.81953037
  0.03958848 -0.1611551   0.10339079 -0.23154226]


In [85]:
print(len(gps_out))
gps_class = [label_dict[pred] for pred in gps_out]
print(len(gps_class))
#print(gps_class)

2940
2940


## Tesla

In [86]:
tsla_out = []
tsla_score = []
for i in range(0, len(tsla_pain), 10):
    tokenized_tsla = tokenizer(tsla_pain[i:i+10], truncation = True, padding = True, return_tensors = "pt")
    tsla_output = model(tokenized_tsla['input_ids'])
    soft_logits_tsla = softmax(np.array(tsla_output.logits.detach().cpu()))
    sentiment_score_tsla = soft_logits_tsla[:,0]-soft_logits_tsla[:,1]
    tsla_pred = np.squeeze(np.argmax(soft_logits_tsla, axis = 1))
    tsla_out.extend(tsla_pred)
    tsla_score.extend(sentiment_score_tsla)
#print(tsla_out) 
print(sentiment_score_tsla)

[0.02990519 0.11737353 0.10327338 0.04139102 0.03391223 0.13352133
 0.06096146 0.1937255  0.3334345  0.1706433 ]


In [87]:
print(len(tsla_out))
tsla_class = [label_dict[pred] for pred in tsla_out]
print(len(tsla_class))
#print(tsla_class)

7600
7600


## JP Morgan Chase

In [88]:
jpm_out = []
jpm_score = []
for i in range(0, len(jpm_pain), 10):
    tokenized_jpm = tokenizer(jpm_pain[i:i+10], truncation = True, padding = True, return_tensors = "pt")
    jpm_output = model(tokenized_jpm['input_ids'])
    soft_logits = softmax(np.array(jpm_output.logits.detach().cpu()))
    sentiment_score = soft_logits[:,0]-soft_logits[:,1]
    jpm_pred = np.squeeze(np.argmax(soft_logits, axis = 1))
    jpm_out.extend(jpm_pred)
    jpm_score.extend(sentiment_score)
#print(jpm_out) 
print(sentiment_score)

[ 0.03611869  0.22267441 -0.35389128  0.01807062 -0.5303399   0.1397978
  0.10331743  0.02986314  0.38122702  0.03484188]


In [89]:
len(jpm_score)

7180

In [90]:
print(len(jpm_out))
jpm_class = [label_dict[pred] for pred in jpm_out]
print(len(jpm_class))
#print(jpm_class)

7180
7180


In [91]:
len(tsla_score)

7600

## Download Stock price 

In [92]:
gps_stock_price  = yf.download('gps', start='2006-11-30', end='2022-02-15', time_interval = 'daily')
tsla_stock_price  = yf.download('tsla', start='2016-11-30', end='2022-02-15', time_interval = 'daily')
jpm_stock_price  = yf.download('jpm,', start='2011-10-20', end='2022-02-15', time_interval = 'daily')


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [93]:
gps_stock_price.tail(10)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-02-01,17.940001,18.440001,17.900000,18.280001,18.087038,9614900
2022-02-02,18.459999,18.459999,17.610001,17.680000,17.493372,61993900
2022-02-03,17.110001,17.900000,17.020000,17.110001,16.929388,12809700
2022-02-04,17.100000,17.299999,16.580000,17.010000,16.830444,7819000
2022-02-07,17.139999,17.500000,16.820000,16.920000,16.741394,6817900
2022-02-08,17.000000,17.580000,16.870001,17.379999,17.196537,6194500
2022-02-09,17.480000,17.540001,17.190001,17.340000,17.156960,8041200
2022-02-10,17.190001,17.879999,17.139999,17.270000,17.087700,7120600
2022-02-11,17.270000,17.270000,15.800000,15.940000,15.771738,15844300


In [94]:
tsla_stock_price.tail(10)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-02-01,935.210022,943.700012,905.000000,931.250000,931.250000,24379400
2022-02-02,928.179993,931.500000,889.409973,905.659973,905.659973,22264300
2022-02-03,882.000000,937.000000,880.520020,891.140015,891.140015,26285200
2022-02-04,897.219971,936.500000,881.169983,923.320007,923.320007,24541800
2022-02-07,923.789978,947.770020,902.710022,907.340027,907.340027,20331500
2022-02-08,905.530029,926.289978,894.799988,922.000000,922.000000,16909700
2022-02-09,935.000000,946.270020,920.000000,932.000000,932.000000,17419800
2022-02-10,908.369995,943.809998,896.700012,904.549988,904.549988,22042300
2022-02-11,909.630005,915.960022,850.700012,860.000000,860.000000,26548600


In [95]:
jpm_stock_price.tail(10)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-02-01,148.690002,151.470001,148.300003,151.149994,150.037857,14584900
2022-02-02,150.500000,150.940002,148.070007,149.940002,148.836761,12441100
2022-02-03,150.000000,151.649994,148.289993,148.699997,147.605881,12474900
2022-02-04,149.100006,153.500000,148.770004,152.559998,151.437485,16654400
2022-02-07,153.270004,153.619995,150.850006,153.070007,151.943741,12572300
2022-02-08,155.000000,158.479996,154.800003,155.949997,154.802536,20411400
2022-02-09,156.589996,157.429993,155.820007,156.600006,155.447769,9131100
2022-02-10,156.660004,159.029999,155.220001,155.949997,154.802536,16455600
2022-02-11,155.610001,158.190002,152.970001,153.919998,152.787476,16830000


In [96]:

gps_stock_price = gps_stock_price.drop(['Open', 'High','Low','Volume', 'Adj Close'], axis = 1)
jpm_stock_price = jpm_stock_price.drop(['Open', 'High','Low','Volume', 'Adj Close'], axis = 1)
tsla_stock_price = tsla_stock_price.drop(['Open', 'High','Low','Volume', 'Adj Close'], axis = 1)


In [97]:
gps_stock_price

,Close
Date,
2006-11-29,19.070000
2006-11-30,18.719999
2006-12-01,18.680000
2006-12-04,18.959999
2006-12-05,18.889999
...,...
2022-02-08,17.379999
2022-02-09,17.340000
2022-02-10,17.270000


## preprocessed and merge data

In [98]:
tsla_timestamp[:5]

['Feb. 15, 2022 at 10:00 a.m. ET',
 'Feb. 15, 2022 at 8:23 a.m. ET',
 'Feb. 15, 2022 at 8:17 a.m. ET',
 'Feb. 15, 2022 at 5:05 a.m. ET',
 'Feb. 14, 2022 at 9:33 p.m. ET']

In [99]:
len(gps_timestamp)

2940

In [100]:
len(tsla_timestamp)

7600

In [101]:
gps_et = []
for e in gps_timestamp:
    gps_et.append(e.replace("ET",""))
jpm_et = []
for e in jpm_timestamp:
    jpm_et.append(e.replace("ET",""))
tsla_et = []
for e in tsla_timestamp:
    tsla_et.append(e.replace("ET",""))

In [102]:
len(gps_et)

2940

In [103]:
gps_at1 = []
for e in gps_et:
    gps_at1.append(e.replace("at",""))
jpm_at1 = []
for e in jpm_et:
    jpm_at1.append(e.replace("at",""))
tsla_at1 = []
for e in tsla_et:
    tsla_at1.append(e.replace("at",""))

In [104]:
len(jpm_at1)

7180

In [105]:
gps_pm1 = []
for e in gps_at1:
    gps_pm1.append(e.replace("p.m.",""))
jpm_pm1 = []
for e in jpm_at1:
    jpm_pm1.append(e.replace("p.m.",""))
tsla_pm1 = []
for e in tsla_at1:
    tsla_pm1.append(e.replace("p.m.",""))

In [106]:
len(jpm_pm1)

7180

In [107]:
gps_am1 = []
for e in gps_pm1:
    gps_am1.append(e.replace("a.m.",""))
jpm_am1 = []
for e in jpm_pm1:
    jpm_am1.append(e.replace("a.m.",""))
tsla_am1 = []
for e in tsla_pm1:
    tsla_am1.append(e.replace("a.m.",""))

In [108]:
gps_commas = []
for e in gps_am1:
    gps_commas.append(e.replace(",",""))
jpm_commas = []
for e in jpm_am1:
    jpm_commas.append(e.replace(",",""))
tsla_commas = []
for e in tsla_am1:
    tsla_commas.append(e.replace(",",""))

In [109]:
gps_fullstops = []
for e in gps_commas:
    gps_fullstops.append(e.replace(".",""))
jpm_fullstops = []
for e in jpm_commas:
    jpm_fullstops.append(e.replace(".",""))
tsla_fullstops = []
for e in tsla_commas:
    tsla_fullstops.append(e.replace(".",""))

In [110]:
tsla_timestamp_final = tsla_fullstops
gps_timestamp_final = gps_fullstops
jpm_timestamp_final = jpm_fullstops


In [111]:
gps_timestamp_final[-5:]

['Jan 10 2022  8:57  ',
 'Jan 7 2022  3:49  ',
 'Dec 19 2021  9:43  ',
 'Dec 14 2021  11:31  ',
 'Dec 13 2021  11:24  ']

## input data into dataframe

In [112]:
#datetime_object = datetime.strptime(gps_timestamp_final, '%b %d %Y %H:%M')
#df_datetime_gps = pd.to_datetime(pd.Series(gps_timestamp_final), format='%b %d %YH:%M')
gps_date = [datetime.strptime(x,'%b %d %Y  %I:%M ') for x in gps_timestamp_final]
#gps_date

In [113]:
tsla_date = [datetime.strptime(x,'%b %d %Y  %I:%M ') for x in tsla_timestamp_final]
jpm_date = [datetime.strptime(x,'%b %d %Y  %I:%M ') for x in jpm_timestamp_final]


In [114]:
jpm_date[-5:]
tsla_date[-5:]

[datetime.datetime(2022, 2, 14, 2, 21),
 datetime.datetime(2022, 2, 14, 11, 28),
 datetime.datetime(2022, 2, 14, 10, 50),
 datetime.datetime(2022, 2, 14, 10, 31),
 datetime.datetime(2022, 2, 14, 10, 15)]

## create a dataframe

In [115]:
print('datetime')
print('tsla date:',len(tsla_date))
print('gps date:',len(gps_date))
print('jpm date:' ,len(jpm_date))

print('stock price') 
print('tsla price:' ,len(tsla_stock_price))
print('gps price:' ,len(gps_stock_price))
print('jpm price:' ,len(jpm_stock_price))

print('sentiment')
print('tsla sentiment:' ,len(tsla_score))
print('gps sentiment:' ,len(gps_score))
print('jpm sentiment:' ,len(jpm_score))


datetime
tsla date: 7600
gps date: 2940
jpm date: 7180
stock price
tsla price: 1312
gps price: 3829
jpm price: 2598
sentiment
tsla sentiment: 7600
gps sentiment: 2940
jpm sentiment: 7180


In [116]:
print(type(tsla_date))

<class 'list'>


In [117]:
gps_score = pd.DataFrame (gps_score, columns = ['sentiment_score'])

In [118]:
tsla_score = pd.DataFrame (tsla_score, columns = ['sentiment_score'])
jpm_score = pd.DataFrame (jpm_score, columns = ['sentiment_score'])

In [119]:
scaler = MinMaxScaler(feature_range=(0, 1))
gps_senti_scaler = scaler.fit_transform(gps_score.values.reshape(-1,1))
tsla_senti_scaler = scaler.fit_transform(tsla_score.values.reshape(-1,1))
jpm_senti_scaler = scaler.fit_transform(jpm_score.values.reshape(-1,1))

In [123]:
gps_date = pd.DataFrame (gps_date, columns = ['Date'])
tsla_date = pd.DataFrame (tsla_date, columns = ['Date'])
jpm_date = pd.DataFrame (jpm_date, columns = ['Date'])

In [126]:
gps_senti_scaler = pd.DataFrame (gps_senti_scaler, columns = ['sentiment_score'])
tsla_senti_scaler = pd.DataFrame (tsla_senti_scaler, columns = ['sentiment_score'])
jpm_senti_scaler = pd.DataFrame (jpm_senti_scaler, columns = ['sentiment_score'])

In [128]:

gps_news_table = pd.concat([gps_date, gps_senti_scaler], axis=1)

tsla_news_table = pd.concat([tsla_date, tsla_senti_scaler], axis=1)

jpm_news_table = pd.concat([jpm_date, tsla_senti_scaler], axis=1)

gps_score = pd.DataFrame (gps_score, columns = ['sentiment_score'])
tsla_score = pd.DataFrame (tsla_score, columns = ['sentiment_score'])
jpm_score = pd.DataFrame (jpm_score, columns = ['sentiment_score'])

scaler = MinMaxScaler(feature_range=(0, 1))
gps_senti_scaler = scaler.fit_transform(gps_score.values.reshape(-1,1))

tsla_senti_scaler = scaler.fit_transform(tsla_score.values.reshape(-1,1))
jpm_senti_scaler = scaler.fit_transform(jpm_score.values.reshape(-1,1))

In [129]:
scaler = MinMaxScaler(feature_range=(0, 1))
gps_scaler = scaler.fit_transform(gps_stock_price.values.reshape(-1,1))
tsla_scaler = scaler.fit_transform(tsla_stock_price.values.reshape(-1,1))
jpm_scaler = scaler.fit_transform(jpm_stock_price.values.reshape(-1,1))


gps_senti_scaler = gps_senti_scaler.tolist()
tsla_senti_scaler = tsla_senti_scaler.tolist()
jpm_senti_scaler =jpm_senti_scaler.tolist()

In [130]:
print(len(gps_news_table))

gps_news_table.isnull().values.any()

2940


False

In [131]:
print(len(tsla_news_table))

tsla_news_table.isnull().values.any()

7600


False

In [132]:
print(len(jpm_news_table))

jpm_news_table.isnull().values.any()

7600


True

In [133]:
gps_news_table

,Date,sentiment_score
0,2022-02-14 05:12:00,0.540694
1,2022-02-14 04:56:00,0.286930
2,2022-02-11 04:56:00,0.290438
3,2022-02-10 04:57:00,0.376313
4,2022-02-09 04:56:00,0.355849
...,...,...
2935,2022-01-10 08:57:00,0.078783
2936,2022-01-07 03:49:00,0.531698
2937,2021-12-19 09:43:00,0.425869
2938,2021-12-14 11:31:00,0.565334


In [134]:
gps_news_table

,Date,sentiment_score
0,2022-02-14 05:12:00,0.540694
1,2022-02-14 04:56:00,0.286930
2,2022-02-11 04:56:00,0.290438
3,2022-02-10 04:57:00,0.376313
4,2022-02-09 04:56:00,0.355849
...,...,...
2935,2022-01-10 08:57:00,0.078783
2936,2022-01-07 03:49:00,0.531698
2937,2021-12-19 09:43:00,0.425869
2938,2021-12-14 11:31:00,0.565334


In [135]:
jpm_news = jpm_news_table.sort_values(by ="Date")
gps_news = gps_news_table.sort_values(by ="Date")
tsla_news = tsla_news_table.sort_values(by ="Date")



In [136]:
jpm_news

,Date,sentiment_score
4119,2010-12-06 00:01:00,0.869969
7159,2011-10-20 05:06:00,0.530473
7158,2011-10-21 00:01:00,0.322909
7156,2011-10-21 04:39:00,0.033791
7155,2011-10-21 06:09:00,0.029845
...,...,...
7595,NaT,0.578116
7596,NaT,0.539988
7597,NaT,0.609752
7598,NaT,0.683167


In [137]:
gps_pure_price = pd.DataFrame(gps_scaler, columns = ['Closing_price'])
tsla_pure_price = pd.DataFrame(tsla_scaler, columns = ['Closing_price'])
jpm_pure_price = pd.DataFrame(jpm_scaler, columns = ['Closing_price'])


In [138]:
gps_pure_price = gps_pure_price.reset_index()
tsla_pure_price = tsla_pure_price.reset_index()
jpm_pure_price = jpm_pure_price.reset_index()

gps_pure_price = gps_pure_price.drop(columns= ['Date'])

tsla_pure_price =tsla_pure_price.drop(columns= ['Date'])
jpm_pure_price =jpm_pure_price.drop(columns= ['Date'])



## merge table 

In [139]:
jpm_stock_price1 = jpm_stock_price.reset_index()
tsla_stock_price1 = tsla_stock_price.reset_index()
gps_stock_price1 = gps_stock_price.reset_index()


In [140]:
jpm_stockprice_date = jpm_stock_price1["Date"]
tsla_stockprice_date = tsla_stock_price1["Date"]
gps_stockprice_date = gps_stock_price1["Date"]


In [141]:
tsla_stockprice_date = pd.DataFrame({ 'Stock_price_date':tsla_stockprice_date.values})
jpm_stockprice_date = pd.DataFrame({ 'Stock_price_date':jpm_stockprice_date.values})
gps_stockprice_date = pd.DataFrame({ 'Stock_price_date':gps_stockprice_date.values})

In [142]:
gps_stockprice_date

,Stock_price_date
0,2006-11-29
1,2006-11-30
2,2006-12-01
3,2006-12-04
4,2006-12-05
...,...
3824,2022-02-08
3825,2022-02-09
3826,2022-02-10
3827,2022-02-11


In [143]:
gps_stock_table =pd.concat([gps_stockprice_date,gps_pure_price], axis=1)
tsla_stock_table =pd.concat([tsla_stockprice_date,tsla_pure_price], axis=1)
jpm_stock_table =pd.concat([jpm_stockprice_date,jpm_pure_price], axis=1)


In [144]:
gps_stock_table

,Stock_price_date,index,Closing_price
0,2006-11-29,0,0.330251
1,2006-11-30,1,0.321733
2,2006-12-01,2,0.320759
3,2006-12-04,3,0.327574
4,2006-12-05,4,0.325870
...,...,...,...
3824,2022-02-08,3824,0.289121
3825,2022-02-09,3825,0.288148
3826,2022-02-10,3826,0.286444
3827,2022-02-11,3827,0.254076


In [145]:
print(tsla_stock_table.isnull().values.any())
print(gps_stock_table.isnull().values.any())
print(jpm_stock_table.isnull().values.any())


False
False
False


In [146]:
gps_stock_table1 = gps_stock_table.set_index('Stock_price_date')
tsla_stock_table1 = tsla_stock_table.set_index('Stock_price_date')
jpm_stock_table1 = jpm_stock_table.set_index('Stock_price_date')


In [147]:
gps_stock_table1


,index,Closing_price
Stock_price_date,,
2006-11-29,0,0.330251
2006-11-30,1,0.321733
2006-12-01,2,0.320759
2006-12-04,3,0.327574
2006-12-05,4,0.325870
...,...,...
2022-02-08,3824,0.289121
2022-02-09,3825,0.288148
2022-02-10,3826,0.286444


In [148]:
gps_stock_table1

,index,Closing_price
Stock_price_date,,
2006-11-29,0,0.330251
2006-11-30,1,0.321733
2006-12-01,2,0.320759
2006-12-04,3,0.327574
2006-12-05,4,0.325870
...,...,...
2022-02-08,3824,0.289121
2022-02-09,3825,0.288148
2022-02-10,3826,0.286444


In [149]:
gps_news_table2 = gps_news_table.sort_values("Date")
tsla_news_table2 = tsla_news_table.sort_values("Date")
jpm_news_table2 = jpm_news_table.sort_values("Date")


In [150]:
gps_news_table2.head(10)

,Date,sentiment_score
2918,2006-11-30 04:20:00,0.061003
2917,2006-11-30 04:27:00,0.055345
2916,2006-11-30 04:42:00,0.042419
2915,2006-11-30 04:51:00,0.487357
2919,2006-11-30 08:41:00,0.919276
2914,2006-12-06 10:38:00,0.466296
2913,2006-12-07 06:23:00,0.117843
2912,2006-12-14 04:41:00,0.949304
2911,2006-12-15 03:42:00,0.523530
2910,2006-12-15 04:52:00,0.681805


In [151]:
jpm_news_table2.head(10)

,Date,sentiment_score
4119,2010-12-06 00:01:00,0.869969
7159,2011-10-20 05:06:00,0.530473
7158,2011-10-21 00:01:00,0.322909
7156,2011-10-21 04:39:00,0.033791
7155,2011-10-21 06:09:00,0.029845
7157,2011-10-21 09:37:00,0.411165
7152,2011-10-24 00:26:00,0.486562
7151,2011-10-24 05:20:00,0.003717
7154,2011-10-24 09:49:00,0.605255
7153,2011-10-24 10:04:00,0.232560


In [152]:
tsla_news_table2.head(10)

,Date,sentiment_score
19,2016-11-04 11:19:00,0.643564
579,2016-11-07 02:05:00,0.562969
1419,2016-11-18 10:24:00,0.521909
919,2016-11-30 05:36:00,0.641331
7575,2016-12-14 00:05:00,0.518197
7579,2016-12-14 03:19:00,0.572672
7578,2016-12-14 07:57:00,0.551298
7577,2016-12-14 09:41:00,0.572820
119,2016-12-14 10:38:00,0.579130
7576,2016-12-14 11:23:00,0.054735


In [153]:
gps_news_table3 = gps_news_table2.set_index('Date')
tsla_news_table3 = tsla_news_table2.set_index('Date')
jpm_news_table3 = jpm_news_table2.set_index('Date')


In [154]:
gps_news_table3

,sentiment_score
Date,
2006-11-30 04:20:00,0.061003
2006-11-30 04:27:00,0.055345
2006-11-30 04:42:00,0.042419
2006-11-30 04:51:00,0.487357
2006-11-30 08:41:00,0.919276
...,...
2022-02-10 04:57:00,0.376313
2022-02-11 04:15:00,0.675075
2022-02-11 04:56:00,0.290438


In [155]:
gps_stock_table1

,index,Closing_price
Stock_price_date,,
2006-11-29,0,0.330251
2006-11-30,1,0.321733
2006-12-01,2,0.320759
2006-12-04,3,0.327574
2006-12-05,4,0.325870
...,...,...
2022-02-08,3824,0.289121
2022-02-09,3825,0.288148
2022-02-10,3826,0.286444


In [156]:
gps_final_table = gps_news_table3.assign(key=gps_news_table3.index.normalize())\
   .merge(gps_stock_table1, left_on='key', right_index=True, how='left').drop('key', 1)

C:\Users\User\AppData\Local\Temp\ipykernel_22868\4207424997.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  .merge(gps_stock_table1, left_on='key', right_index=True, how='left').drop('key', 1)


In [157]:
tsla_final_table = tsla_news_table3.assign(key=tsla_news_table3.index.normalize())\
   .merge(tsla_stock_table1, left_on='key', right_index=True, how='left').drop('key', 1)

C:\Users\User\AppData\Local\Temp\ipykernel_22868\2021002831.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  .merge(tsla_stock_table1, left_on='key', right_index=True, how='left').drop('key', 1)


In [158]:
jpm_final_table = jpm_news_table3.assign(key=jpm_news_table3.index.normalize())\
   .merge(jpm_stock_table1, left_on='key', right_index=True, how='left').drop('key', 1)

C:\Users\User\AppData\Local\Temp\ipykernel_22868\2393391802.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  .merge(jpm_stock_table1, left_on='key', right_index=True, how='left').drop('key', 1)


In [159]:
len(gps_final_table)

2940

In [160]:
jpm_final_table = jpm_final_table.drop(columns= ['index'])

In [161]:
tsla_final_table = tsla_final_table.drop(columns= ['index'])
gps_final_table = gps_final_table.drop(columns= ['index'])

In [162]:
jpm_final_table

,sentiment_score,Closing_price
Date,,
2010-12-06 00:01:00,0.869969,NaN
2011-10-20 05:06:00,0.530473,0.033124
2011-10-21 00:01:00,0.322909,0.035146
2011-10-21 04:39:00,0.033791,0.035146
2011-10-21 06:09:00,0.029845,0.035146
...,...,...
NaT,0.578116,NaN
NaT,0.539988,NaN
NaT,0.609752,NaN


## check for null values

In [163]:
gps_final_table.isnull().any()

sentiment_score    False
Closing_price       True
dtype: bool

In [164]:
tsla_final_table.isnull().any()

sentiment_score    False
Closing_price       True
dtype: bool

In [165]:
jpm_final_table.isnull().any()

sentiment_score    False
Closing_price       True
dtype: bool

## remove rows with null values

In [166]:
gps_for_lstm = gps_final_table.dropna()
tsla_for_lstm = tsla_final_table.dropna()
jpm_for_lstm = jpm_final_table.dropna()


,sentiment_score,Closing_price
Date,,
2011-10-20 05:06:00,0.530473,0.033124
2011-10-21 00:01:00,0.322909,0.035146
2011-10-21 04:39:00,0.033791,0.035146
2011-10-21 06:09:00,0.029845,0.035146
2011-10-21 09:37:00,0.411165,0.035146


In [170]:
gps_for_lstm.isnull().any()

sentiment_score    False
Closing_price      False
dtype: bool

In [171]:
tsla_for_lstm.isnull().any()


sentiment_score    False
Closing_price      False
dtype: bool

In [172]:
jpm_for_lstm.isnull().any()


sentiment_score    False
Closing_price      False
dtype: bool

In [173]:
gps_for_lstm.to_csv('gps_lstm_new.csv', index = True)
tsla_for_lstm.to_csv('tsla_lstm_new.csv', index = True)
jpm_for_lstm.to_csv('jpm_lstm_new.csv', index = True)


## Fill data with previous value

In [174]:
gps_input_dates = pd.date_range(start='30/11/2006', end='14/02/2022', freq = "D")
tsla_input_dates = pd.date_range(start='30/11/2016', end='14/02/2022', freq = "D")
jpm_input_dates = pd.date_range(start='20/10/2011', end='14/02/2022', freq = "D")


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\interactiveshell.py:3361: UserWarning: Parsing '30/11/2006' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\interactiveshell.py:3361: UserWarning: Parsing '14/02/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\interactiveshell.py:3361: UserWarning: Parsing '30/11/2016' in DD/MM/YYYY format. Provide format or specify infer_datetime_fo

In [175]:
jpm_final_table

,sentiment_score,Closing_price
Date,,
2010-12-06 00:01:00,0.869969,NaN
2011-10-20 05:06:00,0.530473,0.033124
2011-10-21 00:01:00,0.322909,0.035146
2011-10-21 04:39:00,0.033791,0.035146
2011-10-21 06:09:00,0.029845,0.035146
...,...,...
NaT,0.578116,NaN
NaT,0.539988,NaN
NaT,0.609752,NaN


In [176]:
gps_for_lstm_chakky = gps_final_table.fillna(method = "ffill")
tsla_for_lstm1 = tsla_final_table.fillna(method = "ffill")
jpm_for_lstm1 = jpm_final_table.fillna(method = "ffill")

In [177]:
jpm_for_lstm_chakky = jpm_for_lstm1.fillna(method = "bfill")


In [178]:
tsla_for_lstm_chakky = tsla_for_lstm1.fillna(method = "bfill")


In [179]:
jpm_for_lstm_chakky.isnull().any()

sentiment_score    False
Closing_price      False
dtype: bool

In [180]:
tsla_for_lstm_chakky.isnull().any()


sentiment_score    False
Closing_price      False
dtype: bool

In [181]:
gps_for_lstm_chakky.isnull().any()


sentiment_score    False
Closing_price      False
dtype: bool

In [182]:
tsla_for_lstm_chakky.head()

,sentiment_score,Closing_price
Date,,
2016-11-04 11:19:00,0.643564,0.001747
2016-11-07 02:05:00,0.562969,0.001747
2016-11-18 10:24:00,0.521909,0.001747
2016-11-30 05:36:00,0.641331,0.001747
2016-12-14 00:05:00,0.518197,0.003303


In [ ]:
gps_for_lstm_chakky.to_csv('gps_lstm_chakky_new.csv', index = True)
tsla_for_lstm_chakky.to_csv('tsla_lstm_chakky_new.csv', index = True)
jpm_for_lstm_chakky.to_csv('jpm_lstm_chakky_new.csv', index = True)

# Save rescale sentiment analysis

#with open('tsla_news_rescale.txt', 'w') as filehandle:
    for listitem in tsla_score:
        filehandle.write('%s\n' % listitem)

with open('jpm_news_rescale.txt', 'w') as filehandle:
    for listitem in jpm_score:
        filehandle.write('%s\n' % listitem)

with open('gps_news_rescale.txt', 'w') as filehandle:
    for listitem in gps_score:
        filehandle.write('%s\n' % listitem)

# Save classified sentiment analysis


with open('tsla_news_score.txt', 'w') as filehandle:
    for listitem in tsla_out:
        filehandle.write('%s\n' % listitem)

with open('jpm_news_score.txt', 'w') as filehandle:
    for listitem in jpm_out:
        filehandle.write('%s\n' % listitem)

with open('gps_news_score.txt', 'w') as filehandle:
    for listitem in gps_out:
        filehandle.write('%s\n' % listitem)


with open('tsla_news_score.txt', 'w') as filehandle:
    for listitem in tsla_out:
        filehandle.write('%s\n' % listitem)

with open('jpm_news_score.txt', 'w') as filehandle:
    for listitem in jpm_out:
        filehandle.write('%s\n' % listitem)

with open('gps_news_score.txt', 'w') as filehandle:
    for listitem in gps_out:
        filehandle.write('%s\n' % listitem)




with open('tsla_news_score_text.txt', 'w') as filehandle:
    for listitem in tsla_class:
        filehandle.write('%s\n' % listitem)

with open('jpm_news_score_text.txt', 'w') as filehandle:
    for listitem in jpm_class:
        filehandle.write('%s\n' % listitem)

with open('gps_news_score_text.txt', 'w') as filehandle:
    for listitem in gps_class:
        filehandle.write('%s\n' % listitem)



with open('tsla_news.txt', 'w') as filehandle:
    for listitem in tsla_final:
        filehandle.write('%s\n' % listitem)


with open('jpm_news.txt', 'w') as filehandle:
    for listitem in jpm_final:
        filehandle.write('%s\n' % listitem)


with open('gps_news.txt', 'w', encoding = 'utf-8') as filehandle:
    for listitem in gps_final:
        filehandle.write('%s\n' % listitem)